In [ ]:
import regu_circle_ellipse
import regu_line
import regu_rectangles
import regu_star
import regu_polygon

def main():
    csv_path = 'problems/problems/frag0.csv'

In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit, leastsq, minimize
from matplotlib import pyplot as plt

def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs

# Straight Line Functions
def line_model(x, m, c):
    return m * x + c

def fit_line(x, y):
    params, _ = curve_fit(line_model, x, y)
    return params

def plot_line(x, y, m, c):
    plt.plot(x, y, 'o', label='Original Data')
    x_fit = np.linspace(np.min(x), np.max(x), 100)
    y_fit = line_model(x_fit, m, c)
    plt.plot(x_fit, y_fit, 'r-', label='Fitted Line')
    plt.legend()
    plt.show()

def main_line(csv_path):
    path_XYs = read_csv(csv_path)
    all_x, all_y = np.hstack([seg[:, 0] for poly in path_XYs for seg in poly]), np.hstack([seg[:, 1] for poly in path_XYs for seg in poly])
    m, c = fit_line(all_x, all_y)
    plot_line(all_x, all_y, m, c)

# Rectangle Functions
def fit_rectangle(x, y):
    return np.min(x), np.max(x), np.min(y), np.max(y)

def main_rectangle(csv_path):
    path_XYs = read_csv(csv_path)
    all_x, all_y = np.hstack([seg[:, 0] for poly in path_XYs for seg in poly]), np.hstack([seg[:, 1] for poly in path_XYs for seg in poly])
    rect_params = fit_rectangle(all_x, all_y)
    print(f"Rectangle Fit Parameters: {rect_params}")

# Polygon Functions
def fit_polygon(x, y, n_sides):
    def polygon_residuals(params):
        xc, yc, radius = params[:3]
        angle_offset = params[3]
        angles = np.linspace(0, 2 * np.pi, n_sides, endpoint=False) + angle_offset
        residuals = np.zeros_like(x)
        for i, angle in enumerate(angles):
            expected_x = xc + radius * np.cos(angle)
            expected_y = yc + radius * np.sin(angle)
            residuals += np.sqrt((x - expected_x)*2 + (y - expected_y)*2)
        return residuals
    initial_guess = np.array([np.mean(x), np.mean(y), np.mean(np.sqrt((x - np.mean(x))*2 + (y - np.mean(y))*2)), 0])
    result = minimize(lambda params: np.sum(polygon_residuals(params)), initial_guess, method='L-BFGS-B')
    return result.x

def main_polygon(csv_path, n_sides=5):
    path_XYs = read_csv(csv_path)
    all_x, all_y = np.hstack([seg[:, 0] for poly in path_XYs for seg in poly]), np.hstack([seg[:, 1] for poly in path_XYs for seg in poly])
    xc, yc, radius, angle_offset = fit_polygon(all_x, all_y, n_sides)
    print(f"Polygon Fit Parameters: Center=({xc}, {yc}), Radius={radius}, Angle Offset={angle_offset}")

# Star Functions
def fit_star(x, y, n_arms):
    def star_residuals(params):
        xc, yc, *arms = params
        angles = np.linspace(0, 2 * np.pi, n_arms, endpoint=False)
        residuals = np.zeros_like(x)
        for i, angle in enumerate(angles):
            expected_distance = arms[i]
            dist = np.sqrt((x - xc)*2 + (y - yc)*2)
            angle_diff = np.arctan2(y - yc, x - xc) - angle
            weight = np.exp(-angle_diff*2 / (2 * (np.pi / n_arms)*2))
            residuals += weight * (dist - expected_distance)**2
        return residuals
    initial_guess = np.array([np.mean(x), np.mean(y)] + [np.mean(np.sqrt((x - np.mean(x))*2 + (y - np.mean(y))*2))] * n_arms)
    result = minimize(lambda params: np.sum(star_residuals(params)), initial_guess, method='L-BFGS-B')
    return result.x

def main_star(csv_path, n_arms=5):
    path_XYs = read_csv(csv_path)
    all_x, all_y = np.hstack([seg[:, 0] for poly in path_XYs for seg in poly]), np.hstack([seg[:, 1] for poly in path_XYs for seg in poly])
    params = fit_star(all_x, all_y, n_arms)
    print(f"Star Fit Parameters: Center=({params[0]}, {params[1]}), Arms={params[2:]}")

# Central Main Function
def main():
    # You can call the different main functions here
    csv_path_line = 'problems/problems/isolated.csv'
    csv_path_rectangle = 'problems/problems/frag01_sol.csv'
    csv_path_polygon = 'problems/problems/frag01_sol.csv'
    csv_path_star = 'problems/problems/frag01_sol.csv'

    main_line(csv_path_line)
    main_rectangle(csv_path_rectangle)
    main_polygon(csv_path_polygon, n_sides=5)
    main_star(csv_path_star, n_arms=5)

# Call the main function directly
main()

FileNotFoundError: problems/problems/frag01_sol.csv not found.

In [6]:
# import regu_circle_ellipse
import regu_line
import regu_rectangles
import regu_star
import regu_polygon
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit, leastsq, minimize
from matplotlib import pyplot as plt

def main(csv_path):
    # Read CSV data
    path_XYs = read_csv(csv_path)
    
    # Combine all segments into one array of x and y coordinates
    all_x, all_y = [], []
    for polyline in path_XYs:
        for segment in polyline:
            x, y = segment[:, 0], segment[:, 1]
            all_x.extend(x)
            all_y.extend(y)
    
    all_x = np.array(all_x)
    all_y = np.array(all_y)
    
    # Fit different shapes and plot results
    print("Fitting Line...")
    m, c = regu_line.fit_line(all_x, all_y)
    regu_line.plot_line(all_x, all_y, m, c)
    mse_line = regu_line.evaluate_fit(all_x, all_y, m, c)
    print(f"Mean Squared Error for Line Fit: {mse_line}")
    
    print("Fitting Rectangle...")
    rectangle_params = regu_rectangles.fit_rectangle(all_x, all_y)
    rounded_params = regu_rectangles.fit_rounded_rectangle(all_x, all_y)
    regu_rectangles.plot_rectangle(all_x, all_y, rectangle_params, rounded_params)
    regu_rectangles.evaluate_fit(all_x, all_y, rectangle_params, rounded_params)
    
    print("Fitting Polygon...")
    xc_poly, yc_poly, radius_poly, angle_offset_poly = regu_polygon.fit_polygon(all_x, all_y, n_sides=5)
    regu_polygon.plot_polygon(all_x, all_y, xc_poly, yc_poly, radius_poly, angle_offset_poly, n_sides=5)
    mse_poly = regu_polygon.evaluate_fit(all_x, all_y, xc_poly, yc_poly, radius_poly, angle_offset_poly, n_sides=5)
    print(f"Mean Squared Error for Polygon Fit: {mse_poly}")
    
    print("Fitting Star...")
    xc_star, yc_star, arms_star = regu_star.fit_star(all_x, all_y, n_arms=5)
    regu_star.plot_star_shape(all_x, all_y, xc_star, yc_star, arms_star, n_arms=5)
    mse_star = regu_star.evaluate_fit(all_x, all_y, xc_star, yc_star, arms_star, n_arms=5)
    print(f"Mean Squared Error for Star Fit: {mse_star}")

# Example usage
csv_path = 'problems/problems/frag0.csv'
main(csv_path)

ModuleNotFoundError: No module named 'regu_line'

In [10]:
from regu_circle_ellipse import regu_circle_ellipse
from regu_line import regu_line
from regu_rectangles import regu_rectangles
from regu_star import regu_star
from regu_polygon import regu_polygon

def main(csv_path):
    # Example usage of functions from each module
    
    # Circle and Ellipse
    try:
        print("Processing with Circle and Ellipse fitting...")
        regu_circle_ellipse.main_combined(csv_path)
    except Exception as e:
        print(f"Error in Circle and Ellipse fitting: {e}")
    
    # Straight Line
    try:
        print("Processing with Line fitting...")
        regu_line.main(csv_path)
    except Exception as e:
        print(f"Error in Line fitting: {e}")
    
    # Rectangles
    try:
        print("Processing with Rectangle fitting...")
        regu_rectangles.main_rectangle_combined(csv_path)
    except Exception as e:
        print(f"Error in Rectangle fitting: {e}")
    
    # Polygon
    try:
        print("Processing with Polygon fitting...")
        regu_polygon.main_combined(csv_path, n_sides=5)  # Example with a 5-sided polygon
    except Exception as e:
        print(f"Error in Polygon fitting: {e}")
    
    # Star Shape
    try:
        print("Processing with Star fitting...")
        regu_star.main_combined(csv_path, n_arms=5)  # Example with 5 arms
    except Exception as e:
        print(f"Error in Star fitting: {e}")

if _name_ == "_main_":
    # Replace with your actual CSV path
    csv_path = 'problems/problems/frag01_sol.csv'
    main(csv_path)

ModuleNotFoundError: No module named 'regu_circle_ellipse'

In [5]:
import numpy as np
from scipy.optimize import curve_fit, leastsq, minimize
from matplotlib import pyplot as plt
import svgwrite

# Function to read CSV file and return the data points
def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs

# Function to fit a straight line
def fit_line(x, y):
    params, _ = curve_fit(lambda x, m, c: m * x + c, x, y)
    m, c = params
    return m, c

# Function to fit a rectangle
def fit_rectangle(x, y):
    return np.min(x), np.max(x), np.min(y), np.max(y)

# Function to fit a polygon
def fit_polygon(x, y, n_sides):
    def polygon_residuals(params):
        xc, yc, radius, angle_offset = params
        angles = np.linspace(0, 2 * np.pi, n_sides, endpoint=False) + angle_offset
        residuals = np.zeros_like(x)
        for i, angle in enumerate(angles):
            expected_x = xc + radius * np.cos(angle)
            expected_y = yc + radius * np.sin(angle)
            residuals += np.sqrt((x - expected_x)*2 + (y - expected_y)*2)
        return residuals
    
    xc_guess, yc_guess = np.mean(x), np.mean(y)
    radius_guess = np.mean(np.sqrt((x - xc_guess)*2 + (y - yc_guess)*2))
    initial_guess = np.array([xc_guess, yc_guess, radius_guess, 0])
    result = minimize(lambda params: np.sum(polygon_residuals(params)), initial_guess, method='L-BFGS-B')
    xc, yc, radius, angle_offset = result.x
    return xc, yc, radius, angle_offset

# Function to fit a star shape
def fit_star(x, y, n_arms):
    def star_residuals(params):
        xc, yc, *arms = params
        angles = np.linspace(0, 2 * np.pi, n_arms, endpoint=False)
        distances = np.sqrt((x - xc)*2 + (y - yc)*2)
        residuals = np.zeros_like(distances)
        for i, angle in enumerate(angles):
            expected_distance = arms[i]
            angle_diff = np.abs(np.arctan2(np.sin(np.arctan2(y - yc, x - xc) - angle),
                                           np.cos(np.arctan2(y - yc, x - xc) - angle)))
            weight = np.exp(-angle_diff*2 / (2 * (np.pi / n_arms)*2))
            residuals += weight * (distances - expected_distance)**2
        return residuals
    
    xc_guess, yc_guess = np.mean(x), np.mean(y)
    arms_guess = np.ones(n_arms) * np.mean(np.sqrt((x - xc_guess)*2 + (y - yc_guess)*2))
    initial_guess = [xc_guess, yc_guess] + list(arms_guess)
    result = minimize(lambda params: np.sum(star_residuals(params)), initial_guess, method='L-BFGS-B')
    xc, yc = result.x[:2]
    arms = result.x[2:]
    return xc, yc, arms

# Function to save the output as an SVG
def save_to_svg(csv_path, shape, params, output_svg='output2.svg'):
    path_XYs = read_csv(csv_path)
    dwg = svgwrite.Drawing(output_svg, profile='tiny')
    
    # Draw the original data points
    for polyline in path_XYs:
        for segment in polyline:
            points = [(x, y) for x, y in segment]
            dwg.add(dwg.polyline(points, stroke='black', fill='none'))
    
    # Draw the fitted shape
    if shape == 'line':
        m, c = params
        x_min = np.min([pt[0] for segment in path_XYs for pt in segment])
        x_max = np.max([pt[0] for segment in path_XYs for pt in segment])
        dwg.add(dwg.line((x_min, m*x_min + c), (x_max, m*x_max + c), stroke='red'))
    elif shape == 'rectangle':
        x_min, x_max, y_min, y_max = params
        dwg.add(dwg.rect((x_min, y_min), (x_max - x_min, y_max - y_min), fill='none', stroke='red'))
    elif shape == 'polygon':
        xc, yc, radius, angle_offset = params
        angles = np.linspace(0, 2 * np.pi, 5, endpoint=False) + angle_offset
        points = [(xc + radius * np.cos(angle), yc + radius * np.sin(angle)) for angle in angles]
        dwg.add(dwg.polygon(points, fill='none', stroke='red'))
    elif shape == 'star':
        xc, yc, arms = params
        angles = np.linspace(0, 2 * np.pi, len(arms), endpoint=False)
        for i, angle in enumerate(angles):
            dwg.add(dwg.line((xc, yc), (xc + arms[i] * np.cos(angle), yc + arms[i] * np.sin(angle)), stroke='red'))
    
    dwg.save()
    print(f'SVG saved to {output_svg}')

# Function to identify and fit the appropriate shape
def identify_and_fit_shape(csv_path):
    path_XYs = read_csv(csv_path)
    
    # Combine all segments into one array of x and y coordinates
    all_x, all_y = [], []
    for polyline in path_XYs:
        for segment in polyline:
            x, y = segment[:, 0], segment[:, 1]
            all_x.extend(x)
            all_y.extend(y)
    
    all_x = np.array(all_x)
    all_y = np.array(all_y)
    
    # Fit line
    m, c = fit_line(all_x, all_y)
    mse_line = np.mean((all_y - (m * all_x + c))**2)
    
    # Fit rectangle
    x_min, x_max, y_min, y_max = fit_rectangle(all_x, all_y)
    mse_rectangle = np.mean(np.minimum(np.abs(all_x - x_min), np.abs(all_x - x_max)) + 
                            np.minimum(np.abs(all_y - y_min), np.abs(all_y - y_max)))
    
    # Fit polygon (example: 5-sided polygon)
    xc, yc, radius, angle_offset = fit_polygon(all_x, all_y, 5)
    angles = np.linspace(0, 2 * np.pi, 5, endpoint=False) + angle_offset
    polygon_residuals = np.array([np.sqrt((all_x - (xc + radius * np.cos(angle)))**2 + 
                                          (all_y - (xc + radius * np.sin(angle)))**2) 
                                  for angle in angles])
    mse_polygon = np.mean(polygon_residuals)
    
    # Fit star shape
    xc_star, yc_star, arms = fit_star(all_x, all_y, 5)
    star_residuals = np.array([np.sqrt((all_x - (xc_star + arms[i] * np.cos(angle)))**2 + 
                                       (all_y - (yc_star + arms[i] * np.sin(angle)))**2) 
                               for i, angle in enumerate(angles)])
    mse_star = np.mean(star_residuals)
    
    # Determine which shape has the lowest mean squared error
    mse_values = {'line': mse_line, 'rectangle': mse_rectangle, 
                  'polygon': mse_polygon, 'star': mse_star}
    best_fit_shape = min(mse_values, key=mse_values.get)
    
    print(f'Best fit shape: {best_fit_shape} with MSE {mse_values[best_fit_shape]}')
    
    if best_fit_shape == 'line':
        save_to_svg(csv_path, 'line', (m, c))
    elif best_fit_shape == 'rectangle':
        save_to_svg(csv_path, 'rectangle', (x_min, x_max, y_min, y_max))
    elif best_fit_shape == 'polygon':
        save_to_svg(csv_path, 'polygon', (xc, yc, radius, angle_offset))
    elif best_fit_shape == 'star':
        save_to_svg(csv_path, 'star', (xc_star, yc_star, arms))

# Example usage:
csv_path = 'problems/problems/frag2.csv'
identify_and_fit_shape(csv_path)

Best fit shape: rectangle with MSE 96.59882074866552
SVG saved to output2.svg


C:\Users\Ayush Pathak\AppData\Local\Temp\ipykernel_15640\539976229.py:42: RuntimeWarning: invalid value encountered in sqrt
  radius_guess = np.mean(np.sqrt((x - xc_guess)*2 + (y - yc_guess)*2))
C:\Users\Ayush Pathak\AppData\Local\Temp\ipykernel_15640\539976229.py:64: RuntimeWarning: invalid value encountered in sqrt
  arms_guess = np.ones(n_arms) * np.mean(np.sqrt((x - xc_guess)*2 + (y - yc_guess)*2))
C:\Users\Ayush Pathak\AppData\Local\Temp\ipykernel_15640\539976229.py:53: RuntimeWarning: invalid value encountered in sqrt
  distances = np.sqrt((x - xc)*2 + (y - yc)*2)


In [9]:
import svgwrite
import cairosvg
import cv2
import numpy as np
from shapely.geometry import Polygon, Point, LineString
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
from skimage import io, color

def svg_to_png(svg_input_path, png_output_path):
    """Convert SVG file to PNG format."""
    cairosvg.svg2png(url=svg_input_path, write_to=png_output_path)
    print(f"Converted SVG to PNG: {png_output_path}")

def process_image(png_path):
    """Load and preprocess the image for shape detection."""
    img = io.imread(png_path)
    img_gray = color.rgb2gray(img)
    img_bin = cv2.threshold(img_gray, 0.5, 1.0, cv2.THRESH_BINARY)[1]
    contours, _ = cv2.findContours(img_bin.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def identify_shapes(contours):
    """Identify and regularize shapes based on contours."""
    shapes = []
    for contour in contours:
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        shape = None

        if len(approx) == 3:
            shape = "triangle"
        elif len(approx) == 4:
            shape = detect_quadrilateral(approx)
        elif len(approx) == 5:
            shape = "pentagon"
        elif len(approx) > 5:
            if is_circle(approx):
                shape = "circle"
            elif is_ellipse(approx):
                shape = "ellipse"
            else:
                shape = "polygon"

        shapes.append((shape, approx))
    return shapes

def detect_quadrilateral(approx):
    """Detects if a quadrilateral is a rectangle or square."""
    (x, y, w, h) = cv2.boundingRect(approx)
    ar = w / float(h)
    if ar >= 0.95 and ar <= 1.05:
        return "square"
    else:
        return "rectangle"

def is_circle(approx):
    """Check if the shape is a circle."""
    area = cv2.contourArea(approx)
    (x, y), radius = cv2.minEnclosingCircle(approx)
    circle_area = np.pi * (radius ** 2)
    return abs(1 - (area / circle_area)) < 0.2

def is_ellipse(approx):
    """Check if the shape is an ellipse."""
    area = cv2.contourArea(approx)
    ellipse = cv2.fitEllipse(approx)
    ellipse_area = np.pi * ellipse[1][0] * ellipse[1][1]
    return abs(1 - (area / ellipse_area)) < 0.2

def create_svg_output(shapes, output_svg_path):
    """Create an SVG output file with detected shapes."""
    dwg = svgwrite.Drawing(output_svg_path, profile='tiny')

    for shape, contour in shapes:
        if shape == "circle":
            (x, y), radius = cv2.minEnclosingCircle(contour)
            dwg.add(dwg.circle(center=(x, y), r=radius, stroke=svgwrite.rgb(0, 0, 0, '%'), fill='none'))
        elif shape == "ellipse":
            ellipse = cv2.fitEllipse(contour)
            dwg.add(dwg.ellipse(center=(ellipse[0][0], ellipse[0][1]), r=(ellipse[1][0]/2, ellipse[1][1]/2), stroke=svgwrite.rgb(0, 0, 0, '%'), fill='none'))
        elif shape == "rectangle":
            x, y, w, h = cv2.boundingRect(contour)
            dwg.add(dwg.rect(insert=(x, y), size=(w, h), stroke=svgwrite.rgb(0, 0, 0, '%'), fill='none'))
        elif shape == "polygon":
            points = [(p[0][0], p[0][1]) for p in contour]
            dwg.add(dwg.polygon(points, stroke=svgwrite.rgb(0, 0, 0, '%'), fill='none'))

    dwg.save()
    print(f"SVG output created: {output_svg_path}")

def main(svg_input_path, csv_input_path, output_svg_path, intermediate_png_path):
    # Step 1: Convert SVG to PNG
    svg_to_png(svg_input_path, intermediate_png_path)

    # Step 2: Process the PNG image
    contours = process_image(intermediate_png_path)

    # Step 3: Identify and regularize shapes
    shapes = identify_shapes(contours)

    # Step 4: Create SVG output with the identified shapes
    create_svg_output(shapes, output_svg_path)

# Example usage:
svg_input_path = 'problems/problems/frag0.svg'
csv_input_path = 'problems/problems/frag0.csv'  # Currently unused but can be incorporated for additional functionality.
output_svg_path = 'output4.svg'
intermediate_png_path = 'intermediate.png'

main(svg_input_path, csv_input_path, output_svg_path, intermediate_png_path)

ModuleNotFoundError: No module named 'skimage'

In [10]:
import numpy as np
from scipy.optimize import curve_fit, minimize
from matplotlib import pyplot as plt
import svgwrite

# Function to read CSV file and return the data points
def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs

# Function to fit a straight line
def line_model(x, m, c):
    return m * x + c

def fit_line(x, y):
    params, _ = curve_fit(line_model, x, y)
    return params

def plot_line(x, y, m, c):
    plt.figure(figsize=(8, 8))
    plt.plot(x, y, 'o', label='Original Data')
    x_fit = np.linspace(np.min(x), np.max(x), 100)
    y_fit = line_model(x_fit, m, c)
    plt.plot(x_fit, y_fit, 'r-', label='Fitted Line')
    plt.title("Line Fitting")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend()
    plt.axis('equal')
    plt.show()

# Function to fit a rectangle
def fit_rectangle(x, y):
    return np.min(x), np.max(x), np.min(y), np.max(y)

# Function to fit a polygon with a specified number of sides
def fit_polygon(x, y, n_sides):
    def polygon_residuals(params):
        xc, yc, radius, angle_offset = params
        angles = np.linspace(0, 2 * np.pi, n_sides, endpoint=False) + angle_offset
        residuals = np.zeros_like(x)
        for i, angle in enumerate(angles):
            expected_x = xc + radius * np.cos(angle)
            expected_y = yc + radius * np.sin(angle)
            residuals += np.sqrt((x - expected_x)*2 + (y - expected_y)*2)
        return residuals

    xc_guess, yc_guess = np.mean(x), np.mean(y)
    radius_guess = np.mean(np.sqrt((x - xc_guess)*2 + (y - yc_guess)*2))
    initial_guess = np.array([xc_guess, yc_guess, radius_guess, 0])
    result = minimize(lambda params: np.sum(polygon_residuals(params)), initial_guess, method='L-BFGS-B')
    xc, yc, radius, angle_offset = result.x
    return xc, yc, radius, angle_offset

# Function to fit a star shape with a specified number of arms
def fit_star(x, y, n_arms):
    def star_residuals(params):
        xc, yc, *arms = params
        angles = np.linspace(0, 2 * np.pi, n_arms, endpoint=False)
        distances = np.sqrt((x - xc)*2 + (y - yc)*2)
        residuals = np.zeros_like(distances)
        for i, angle in enumerate(angles):
            expected_distance = arms[i]
            angle_diff = np.abs(np.arctan2(np.sin(np.arctan2(y - yc, x - xc) - angle),
                                           np.cos(np.arctan2(y - yc, x - xc) - angle)))
            weight = np.exp(-angle_diff*2 / (2 * (np.pi / n_arms)*2))
            residuals += weight * (distances - expected_distance)**2
        return residuals

    xc_guess, yc_guess = np.mean(x), np.mean(y)
    arms_guess = np.ones(n_arms) * np.mean(np.sqrt((x - xc_guess)*2 + (y - yc_guess)*2))
    initial_guess = [xc_guess, yc_guess] + list(arms_guess)
    result = minimize(lambda params: np.sum(star_residuals(params)), initial_guess, method='L-BFGS-B')
    xc, yc = result.x[:2]
    arms = result.x[2:]
    return xc, yc, arms

# Function to save the fitted shape and original points as an SVG file
def save_to_svg(csv_path, shape, params, output_svg='output.svg'):
    path_XYs = read_csv(csv_path)
    dwg = svgwrite.Drawing(output_svg, profile='tiny')
    
    for polyline in path_XYs:
        for segment in polyline:
            points = [(x, y) for x, y in segment]
            dwg.add(dwg.polyline(points, stroke='black', fill='none'))
    
    if shape == 'line':
        m, c = params
        x_min = np.min([pt[0] for segment in path_XYs for pt in segment])
        x_max = np.max([pt[0] for segment in path_XYs for pt in segment])
        dwg.add(dwg.line((x_min, m*x_min + c), (x_max, m*x_max + c), stroke='red'))
    elif shape == 'rectangle':
        x_min, x_max, y_min, y_max = params
        dwg.add(dwg.rect((x_min, y_min), (x_max - x_min, y_max - y_min), fill='none', stroke='red'))
    elif shape == 'polygon':
        xc, yc, radius, angle_offset = params
        angles = np.linspace(0, 2 * np.pi, 5, endpoint=False) + angle_offset
        points = [(xc + radius * np.cos(angle), yc + radius * np.sin(angle)) for angle in angles]
        dwg.add(dwg.polygon(points, fill='none', stroke='red'))
    elif shape == 'star':
        xc, yc, arms = params
        angles = np.linspace(0, 2 * np.pi, len(arms), endpoint=False)
        for i, angle in enumerate(angles):
            next_angle = angle + np.pi / len(arms)
            dwg.add(dwg.line((xc + arms[i] * np.cos(angle), yc + arms[i] * np.sin(angle)),
                             (xc + arms[i] * np.cos(next_angle), yc + arms[i] * np.sin(next_angle)),
                             stroke='red', stroke_width=2))
    
    dwg.save()
    print(f'SVG saved to {output_svg}')

# Function to identify and fit the appropriate shape
def identify_and_fit_shape(csv_path):
    path_XYs = read_csv(csv_path)
    all_points = np.vstack([segment for polyline in path_XYs for segment in polyline])
    x, y = all_points[:, 0], all_points[:, 1]
    
    m_line, c_line = fit_line(x, y)
    line_error = np.mean((y - (m_line * x + c_line))**2)
    
    x_min, x_max, y_min, y_max = fit_rectangle(x, y)
    rectangle_error = np.mean((y - np.clip(y, y_min, y_max))*2 + (x - np.clip(x, x_min, x_max))*2)
    
    n_sides = 5
    xc_polygon, yc_polygon, radius_polygon, angle_offset_polygon = fit_polygon(x, y, n_sides)
    polygon_error = np.mean([np.min(np.sqrt((x - xc_polygon)*2 + (y - yc_polygon)*2) - radius_polygon)])
    
    n_arms = 5
    xc_star, yc_star, arms = fit_star(x, y, n_arms)
    star_error = np.mean([(np.sqrt((x - xc_star)**2 + (y - yc_star)**2) - arms[i % n_arms])**2
                      for i in range(len(x))])

    
    errors = {'line': line_error, 'rectangle': rectangle_error,
              'polygon': polygon_error, 'star': star_error}
    best_shape = min(errors, key=errors.get)
    
    print(f'Best shape: {best_shape} with error: {errors[best_shape]}')
    
    if best_shape == 'line':
        save_to_svg(csv_path, 'line', (m_line, c_line))
    elif best_shape == 'rectangle':
        save_to_svg(csv_path, 'rectangle', (x_min, x_max, y_min, y_max))
    elif best_shape == 'polygon':
        save_to_svg(csv_path, 'polygon', (xc_polygon, yc_polygon, radius_polygon, angle_offset_polygon))
    elif best_shape == 'star':
        save_to_svg(csv_path, 'star', (xc_star, yc_star, arms))

# Example usage:
csv_path = 'problems/problems/frag2.csv'
identify_and_fit_shape(csv_path)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2565601440.py, line 139)